# Get data from OLMO
This workbook includes the code to query data directly from the OceanLab Marine Observatory (OLMO) Database, using the InfluxDB Client.
within Iliad we are planning to implement a standardised API, this not in place, yet.

## Imports
You will need the dotenv package to read the credentials for the database and the influxdb_client to access the database. We use pandas to export the data in a decent format that is then easy convertable to JSON, CSV or netCDF.

In [13]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import influxdb_client
from datetime import datetime
#from dotenv import dotenv_values

# database tables
The OLMO database is optimised for time series. A table is called 'measurement' and different databases are called 'buckets'. Our openly shared data is in the bucket called "oceanlab". Our more experimental or project restricted data is in the bucket "example".

OLMO consist of several sites with sensors, in this example we look at the site called "Munkholmen" and the sensor called "CTD". The time series data from this sensor is stored in tables called "ctd_<variable>_munkholmen.

The position of that sensor is the position of the location and stored in the table "meteo_position_munkholmen". The water depth is in the table "ctd_depth_munkholmen". Also these data are time series as the buoy (sensor platform) is moving in the water.

Just for fun we also take air temperature, wind speed and wind direction measured at the location.

In [8]:
# tables (measurements) of interest:
'''

---- ctd_depth_munkholmen:
                       time approved data_level    depth           edge_device   platform sensor   unit
2022-10-18T09:06:39.032950Z       no  processed 1.207182 munkholmen_topside_pi munkholmen    ctd metres

---- ctd_salinity_munkholmen:
                       time approved data_level           edge_device   platform  salinity sensor unit
2022-10-18T09:06:39.032950Z       no        raw munkholmen_topside_pi munkholmen   28.2052    ctd none

---- ctd_temperature_munkholmen:
                       time approved data_level           edge_device   platform sensor  temperature            unit
2022-10-18T09:06:39.032950Z       no        raw munkholmen_topside_pi munkholmen    ctd        8.969 degrees_celcius

---- meteo_position_munkholmen:
                time approved data_level edge_device  latitude  longitude   platform sensor    unit
2023-08-28T22:00:16Z      yes        raw         cr6 63.457478  10.372295 munkholmen    gps degrees

---- meteo_temperature_munkholmen:
                time approved data_level edge_device   platform sensor  temperature            unit
2023-08-28T22:00:16Z      yes        raw         cr6 munkholmen    gps         13.7 degrees_celsius

---- meteo_wind_direction_munkholmen:
                time approved data_level edge_device   platform sensor    unit  wind_direction
2023-08-28T22:00:16Z      yes        raw         cr6 munkholmen    gps degrees           245.0

---- meteo_wind_speed_munkholmen:
                time approved data_level edge_device   platform sensor              unit  wind_speed
2023-08-28T22:00:16Z      yes        raw         cr6 munkholmen    gps metres_per_second        1.94
'''

# dictionary with CF standard variable name : data table name
data_tables={
    'sea_water_temperature':'ctd_temperature_munkholmen', 
    'sea_water_salinity':'ctd_salinity_munkholmen', 
    'sensor_depth':'ctd_depth_munkholmen',
    'sensor_position': 'meteo_position_munkholmen',
    'wind_speed':'meteo_wind_speed_munkholmen', 
    'wind_direction':'meteo_wind_direction_munkholmen', 
    'air_temperature':'meteo_temperature_munkholmen', 
}

In [9]:

def query_to_df(url, token, query):
    #queries database at url with query and returns pandas DataFrame
    # :url: url to database
    # :token: username & password
    # :query: query string in InfluxDbClient format
    # returns: pandas DataFrame
    
    with influxdb_client.InfluxDBClient(url=url, token=token) as client:
        df = client.query_api().query_data_frame(query)
    return df

def define_query_since(bucket, measurement, timespan):
    # Set up query, in this example data since today-{timespan} from table {measurement}
    # :bucket: database name string
    # :measurement: name of table containing time series
    # :timespan: startdate as string, e.g. '2023-05-01T00:00:00Z'
    # returns query string in InfluxDbClient format

    query = f'''from(bucket:"{bucket}")
 |> range(start:-{timespan})
 |> filter(fn:(r) => r._measurement == "{measurement}")
 |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")'''

    return query

def define_query_interval(bucket, measurement, start_date, end_date):
    # Set up query, in this example data since start_date to end_date from table {measurement}
    # :bucket: database name string
    # :measurement: name of table containing time series
    # :start_date: startdate as string, e.g. '2023-05-01T00:00:00Z'
    # :end_date: end_date as string, e.g. '2023-05-02T00:00:00Z'
    # returns query string in InfluxDbClient format
    
    query = f'''from(bucket:"{bucket}")
 |> range(start: {start_date} , stop: {end_date})
 |> filter(fn:(r) => r._measurement == "{measurement}")
 |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")'''

    return query

In [20]:
def get_data_from_munkholmen():
    bucket = "oceanlab"

    data_dict=dict()
    for variable, data_table in data_tables.items(): #='ctd_temperature_munkholmen'
        #timespan='240m' #240 minutes datetime.timedelta(hours=-1)
        #query=define_query_since(bucket, data_table, timespan)
        start=datetime(2023, 5, 1, 15)
        end=datetime(2023, 5, 1, 20)
        query=define_query_interval(bucket, data_table, '2023-05-01T00:00:00Z', '2023-05-02T00:00:00Z')

        token = f"waterquality_iliad:SDr2YNsxRyShvB4Z"
        url = "https://oceanlab.azure.sintef.no:8086"

        print(variable)
        try: 
            data_df=query_to_df(url, token, query)
            data_df.to_csv(f'munkhomen__{variable}.csv')
        except:
            print('Sth. went wrong. See error log for details.')
        
        data_dict[data_table]=data_df
    return data_dict

olmo_data=get_data_from_munkholmen()
olmo_data


sea_water_temperature
sea_water_salinity
sensor_depth
sensor_position
wind_speed
wind_direction
air_temperature


{'ctd_temperature_munkholmen':         result  table                            _time  \
 0      _result      0 2023-05-01 02:00:30.182941+00:00   
 1      _result      0 2023-05-01 02:00:31.547371+00:00   
 2      _result      0 2023-05-01 02:00:31.753590+00:00   
 3      _result      0 2023-05-01 02:00:31.994409+00:00   
 4      _result      0 2023-05-01 02:00:32.250696+00:00   
 ...        ...    ...                              ...   
 46361  _result      0 2023-05-01 23:25:20.313672+00:00   
 46362  _result      0 2023-05-01 23:25:20.542116+00:00   
 46363  _result      0 2023-05-01 23:25:20.799993+00:00   
 46364  _result      0 2023-05-01 23:25:21.053606+00:00   
 46365  _result      0 2023-05-01 23:25:21.336647+00:00   
 
                          _start                     _stop  \
 0     2023-05-01 00:00:00+00:00 2023-05-02 00:00:00+00:00   
 1     2023-05-01 00:00:00+00:00 2023-05-02 00:00:00+00:00   
 2     2023-05-01 00:00:00+00:00 2023-05-02 00:00:00+00:00   
 3     2023-

# Get particle data from camera
note: the camera has not been deployed since May (?)

Particles in the water are recorded by a camera device called SilCam. While we are also looking into particle classification in the future we are currently looking at particle sizes only. The data is stored in a database table called "silcam_total_volume_concentration_munkholmen".
The columns containing the measurements denote the partical diameter as bins 2.72, 3.2096, 3.787328, 4.46904704, i.e. (0:2.72], (2.72:3.2096], etc.

In [17]:
def stokes_settling_speed(d, g=9.81, mu=1.4e-3, rhos=3300.0, rhow=1027.0):
    '''Stokes' law, settling speed from particle diameter [m/s]'''
    #g = 9.81 # m / s**2
    #mu = 1.08e-3 # Pa s = kg / (m * s)
    #delta_rho = (3300.0 - 1025.0) # kg/m**3
    delta_rho = rhos - rhow

    return g * delta_rho / (18.0 * mu) * d**2

def store_silcam_data_nicely(silcam):
    # Store SilCam data in csv file
    #:silcam: pandas Dataframe from influx DB table silcam_total_volume_concentration_munkholmen
    
    silcam_export = silcam.set_index('_time').iloc[:, 12:]
    silcam_export.columns = np.round(silcam_export.columns.astype(float), 3)
    silcam_export = silcam_export.sort_index(axis=1)
    silcam_export.to_csv('munkholmen__silcam-particle-volume-concentration-microlitre-per-litre-per-micrometer-bin.csv')
    

def get_silcam_data(start_date, end_date):
    '''Retrieve Munkholmen buoy SilCam particle volume concentration from InfluxDB'''
    
    DATABASE = 'example'
    RETPOLICY = 'autogen'
    bucket = f"{DATABASE}/{RETPOLICY}"
    url = "https://oceanlab.azure.sintef.no:8086"

    query = f'''
        from(bucket:"{bucket}")
            |> range(start: {start_date} , stop: {end_date})
            |> filter(fn:(r) => r._measurement == "silcam_total_volume_concentration_munkholmen")
            |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
        '''
    try: 
        with influxdb_client.InfluxDBClient(url=url, token=f'{USER}:{PASSWORD}') as client:
            df = client.query_api().query_data_frame(query)
            store_silcam_data_nicely(df)
        return df 
    except: 
        print('Sth. went wrong. See error log for details.')
 

In [18]:
silcam_data = get_silcam_data('2023-02-08T00:00:00Z', '2023-02-10T00:00:00Z')

Sth. went wrong. See error log for details.


In [19]:
bins= np.sort(np.array(silcam.columns.sort_values()[:-12]).astype(float))
bins

NameError: name 'silcam' is not defined

In [5]:
silcam.head()

NameError: name 'silcam' is not defined

In [111]:
# Check unit and measurement type
silcam['unit'][0], silcam['_measurement'][0]

('microlitres_per_litre', 'silcam_total_volume_concentration_munkholmen')

# Access to Munkholmen data for SensorThings API in Iliad - Water Quality Pilot
The water quality pilot is implelemneted in OceanLab, a research infrastructure owned by Iliad partners SINTEF OCean and NTNU.

OceanLab curently consist of 4 'nodes', which are thematic areas within ocean science and industries (Subsea Robotics, Aquaculture, Autonomous Shipping and Ocean Observatory).
Each node has one or several 'geographic locations'.
Each geographic location  has one or several 'sensor platforms'.
Each sensor platform can have one or several 'sensors'.
Each sensor collects one or several 'observations' of interest.

## The Iliad water quality pilot
The water quality pilot will use data from 2 nodes in OceanLab
1. Ocean Lab Observatory
2. Ocean Lab Subsea Robotics

### Ocean Lab Marine Observatory (OLMO)
The Ocean Lab Observatory consists of (currently) two geographic locations:
1. Munkholmen 
2. Ingdalen 
At each location there is a sensor platform (buoy). All data from the Ocean Lab Observatory is transfered via telemetry into an influx database (time series database). No data indicates that a sensor is not deployed or that sth. is malfunctioning.

#### Munkholmen buoy
The Munkholmen buoy is deployed at 63.457 N, 10.372 E. This position varies in the sea water due to exposure to eind, waves and current and is therefore stored as a time series in the influx database table meteo_position_munkholmen (below table column headers and example data):
time approved data_level edge_device  latitude  longitude   platform sensor    unit
2023-08-03T22:00:14Z      yes        raw         cr6 63.457466  10.372307 munkholmen    gps degrees

The Munkholmen buoy has several sensors that collect data to be used with Iliad:
1. CTD (https://en.wikipedia.org/wiki/CTD_(instrument))
2. ADCP (https://en.wikipedia.org/wiki/Acoustic_Doppler_current_profiler)
3. SilCam (if deployed) (https://github.com/SINTEF/PySilCam/wiki)

##### CTD
The CTD collects 
1. Sea water temperature and 
2. Sea water conductivity data, from conductivity, sea water salinity is derived. 
3. Sea water density
4. Ambient pressure to determine sea water depth of the measurement

The sensor is deployed at a depth of ca. 2m. This depths varies due to waves and currents and is determined by a pressor sensor in the device. Pressure and depth are stored as time series in the influx database   

table ctd_pressure_munkholmen (below table column headers and example data):   
time approved data_level           edge_device   platform  pressure sensor unit   
2022-08-11T23:00:30.003363Z       no        raw munkholmen_topside_pi munkholmen     1.072    ctd none   

table ctd_depth_munkholmen (below table column headers and example data):   
time approved data_level    depth           edge_device   platform sensor   unit   
2022-08-11T23:00:30.003363Z       no  processed 1.061607 munkholmen_topside_pi munkholmen    ctd metres   

Data tables from this sensor are:   
ctd_conductivity_munkholmen:   
time approved  conductivity data_level           edge_device   platform sensor               unit   
2022-08-11T23:00:30.003363Z       no       2.49332        raw munkholmen_topside_pi munkholmen    ctd siemens_per_metre   

ctd_density_munkholmen:   
time approved data_level     density           edge_device   platform sensor                      unit   
2022-08-11T23:00:30.003363Z       no  processed 1013.838487 munkholmen_topside_pi munkholmen    ctd  kilograms_per_cubic_metre   

ctd_salinity_munkholmen:   
time approved data_level           edge_device   platform  salinity sensor unit   
2022-08-11T23:00:30.003363Z       no        raw munkholmen_topside_pi munkholmen   19.2053    ctd none   

ctd_temperature_munkholmen:   
time approved data_level           edge_device   platform sensor  temperature            unit   
2022-08-11T23:00:30.003363Z       no        raw munkholmen_topside_pi munkholmen    ctd      14.9741 degrees_celcius   

###### Sea water temperature at Munkholmen
Sea water temperature is retrieved from the table ctd_temperature_munkholmen. 
https://cfconventions.org/Data/cf-standard-names/current/build/cf-standard-name-table.html:   
sea_water_temperature   
Sea water temperature is the in situ temperature of the sea water. To specify the depth at which the temperature applies use a vertical coordinate variable or scalar coordinate variable. 

It is used in the pilot for plotting (data from past X days) and in a model for particle transport, where it is needed in the standardised format   
*sea_water_temperature(time, depth, latitude, longitude)*   

API calls for sea water temperature include:
- get temperature -> returns all available data (as netCDF file for download?)
- get temperature since {datetime}{resolution} -> returns all available data measured since {datetime} with resolution optional (minute, hourly, daily, monthly average) (as netCDF file for download?)
- get temperature between {datetime} and {datetime}{resolution} -> returns all available data measured between {datetime} and {datetime} with resolution optional (minute, hourly, daily, monthly average) (as netCDF file for download?)

### Ocean Lab Subsea Robotics
- later - 